In [34]:
# load results
import numpy as np

alexnet_predictions = np.load("alexnet_predictions.npz")["arr_0"]
alexnet_predictions

array([[3.94779863e-03, 2.13177186e-02, 2.82587498e-01, ...,
        2.13121027e-02, 5.10923739e-04, 7.79809547e-04],
       [3.17397341e-02, 2.27939729e-02, 5.96201345e-02, ...,
        2.25043856e-02, 8.88946839e-03, 9.63680912e-03],
       [1.03488244e-01, 1.84415221e-01, 7.55974829e-01, ...,
        6.58041053e-03, 3.66960512e-03, 1.04460865e-02],
       ...,
       [2.55904216e-02, 2.07660664e-02, 1.19247288e-01, ...,
        9.75004956e-03, 8.14527366e-03, 3.96740716e-03],
       [1.05909910e-02, 5.22454968e-03, 8.89304131e-02, ...,
        1.62097216e-02, 8.68206099e-03, 3.23032681e-03],
       [5.39686829e-02, 1.37956887e-02, 5.94881102e-02, ...,
        1.52153283e-01, 3.16583016e-03, 3.53303924e-03]], dtype=float32)

In [35]:
import pandas as pd


from dataset import PosterDataset

df = pd.read_csv(PosterDataset.csv_file)
true_values = df.iloc[:, -20:-1].values
genres = df.columns[-20:-1]
genres = list(map(lambda x: x[1:], genres))
single_genres = df.genre_count == 1

In [36]:
from sklearn.model_selection import train_test_split

train_indices, test_indices = train_test_split(
    np.arange(len(df)), test_size=0.2, random_state=42
)

In [37]:
test_mask = np.isin(np.arange(len(df)), test_indices)
single_genres &= test_mask

In [45]:
single_prediction

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [48]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, roc_auc_score

# calculate accuracy
single_prediction = (
    alexnet_predictions[single_genres]
    == alexnet_predictions.max(axis=1)[single_genres][:, None]
)


roc_auc_score(true_values[single_genres], single_prediction, average="macro"), f1_score(
    true_values[single_genres], single_prediction, average="macro"
), precision_score(
    true_values[single_genres], single_prediction, average="macro"
), accuracy_score(
    true_values[single_genres], single_prediction
),

/Users/borisleung/anaconda3/envs/stat4012proj/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.5450274581354139,
 0.13382643304472486,
 0.22662229906168885,
 0.42892909805849105)

### 3 Single Label


In [39]:
three_columns = ["Action", "Romance", "Thriller"]
three_genres = np.isin(genres, three_columns)
three_genres_index_mask = (
    true_values[:, three_genres].sum(axis=1) >= 1
) & single_genres
three_genres_index_mask.sum()

3275

In [44]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, roc_auc_score

# calculate accuracy
three_single_prediction = alexnet_predictions[three_genres_index_mask,][:, three_genres]
three_single_prediction_classes = (
    three_single_prediction == three_single_prediction.max(axis=1)[:, None]
)

roc_auc_score(
    true_values[three_genres_index_mask,][:, three_genres],
    three_single_prediction_classes,
    average="macro",
), f1_score(
    true_values[three_genres_index_mask,][:, three_genres],
    three_single_prediction_classes,
    average="macro",
), precision_score(
    true_values[three_genres_index_mask,][:, three_genres],
    three_single_prediction_classes,
    average="macro",
), accuracy_score(
    true_values[three_genres_index_mask,][:, three_genres],
    three_single_prediction_classes,
)

(0.715957695221714, 0.6153952135839104, 0.6279016787731134, 0.6177099236641221)

In [41]:
three_single_prediction_classes.sum(axis=0) / three_single_prediction.shape[0]

array([0.25160305, 0.42259542, 0.32580153])